We are finding that June and July and sometimes May are consistently the best months for a theatrical release. December and to a lesser extend, November and January. Then February, March, and April are dead months. Lastly August, September, and October are ok.

First, I will take the "Earliest Release Data" column and parse out into "country of earliest release" and "Earliest Release DateTime"
Then I will use the month of the date time object to split into 1 of 4 categories as discussed above.
Then I will compare the datetime to the datetimes of federal holidays and give a 1 if near the date time of a federal holiday, 0 otherwise, were near is defined in parametes

In [2]:
from datetime import datetime
import pandas as pd


In [3]:
test = "January 31, 2014 (Spain)"

In [4]:
def to_dt(release_date):
    try:
        date_l = release_date.split("(")[0]
        # Sample string representing a date
        date_string = date_l.strip()

        # Convert the string to a datetime object
        datetime_object = datetime.strptime(date_string, '%B %d, %Y')
        return datetime_object
    except:
        return pd.NaT

In [5]:
df = pd.read_csv("./merged_data_mod_scores.csv")
df["Release Date Datetime"] = df.apply(lambda x: to_dt(x["Earliest Release Date"]), axis=1)

In [6]:
def split_by_month(date):
    try:
        if date.month in {5,6,7}:
            return "MJJ_3"
        elif date.month in {8,9,10}:
            return "ASO_4"
        elif date.month in {11,12,1}:
            return "NDJ_1"
        elif date.month in {2,3,4}:
            return "FMA_2"
    except:
        return pd.NA

In [7]:
df["Season"] = df.apply(lambda x: split_by_month(x["Release Date Datetime"]), axis=1)

In [8]:
df["Season"]

0       NDJ_1
1       ASO_4
2       NDJ_1
3       FMA_2
4       FMA_2
        ...  
9801    ASO_4
9802    MJJ_3
9803    ASO_4
9804    ASO_4
9805    ASO_4
Name: Season, Length: 9806, dtype: object

In [9]:
from datetime import date 
import holidays 
  
# Select country 
uk_holidays = holidays.UnitedStates() 
print(uk_holidays)
  
# If it is a holidays then it returns True else False
print('01-01-2019' in uk_holidays) 
 
  
# What holidays is it? 

print(uk_holidays)


{'country': US, 'expand': True, 'language': None, 'market': None, 'observed': True, 'subdiv': None, 'years': set()}
True
{datetime.date(2019, 1, 21): 'Martin Luther King Jr. Day', datetime.date(2019, 2, 18): "Washington's Birthday", datetime.date(2019, 10, 14): 'Columbus Day', datetime.date(2019, 1, 1): "New Year's Day", datetime.date(2019, 5, 27): 'Memorial Day', datetime.date(2019, 7, 4): 'Independence Day', datetime.date(2019, 9, 2): 'Labor Day', datetime.date(2019, 11, 11): 'Veterans Day', datetime.date(2019, 11, 28): 'Thanksgiving', datetime.date(2019, 12, 25): 'Christmas Day'}


In [10]:
from datetime import datetime, timedelta
import holidays

def is_near_holiday(input_date, country='US'):
    """
    Check if a given date is within a week prior to or 5 days after a US holiday.

    Parameters:
    input_date (datetime): The date to check.
    country (str): The country code for the holidays. Default is 'US'.

    Returns:
    bool: True if the date is near a holiday, False otherwise.
    """
    us_holidays = holidays.CountryHoliday(country)

    # Check a week before and 5 days after each holiday in the year of the input date
    for date in us_holidays.keys():
        if input_date >= date - timedelta(days=7) and input_date <= date + timedelta(days=5):
            return True

    return False

# Example usage
example_date = datetime(2023, 1, 1)  # New Year's Day in 2023
print(is_near_holiday(example_date))


False


In [11]:
df["Holiday"] = df.apply(lambda x: is_near_holiday(x["Release Date Datetime"]), axis=1)

In [14]:
df["Release Date Datetime"]

0      2014-01-31
1      2007-09-30
2      2013-01-18
3      2014-04-11
4      2013-02-22
          ...    
9801   2010-08-27
9802   2010-06-17
9803   2010-09-03
9804   2011-08-26
9805   2010-08-12
Name: Release Date Datetime, Length: 9806, dtype: datetime64[ns]